## Video

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
#import PyQt5
%matplotlib qt
import pickle
import os.path
%matplotlib inline

# load pickled distortion matrix
with open('wide_dist_pickle.p', mode='rb') as f:
    dist_pickle = pickle.load(f)
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]

In [2]:
img_size = 1280, 720
midpoint = 640

In [3]:
def undist_image(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    #undist = cv2.cvtColor(undist, cv2.COLOR_BGR2RGB)
    return undist

In [4]:
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return color_binary, combined_binary

In [5]:
# source and distination for perspective tranfsorm
src = np.float32(
        [[img_size[0]/2 - 60, img_size[1]/2 + 100],
         [img_size[0]/6 - 25, img_size[1]],
         [img_size[0]*5/6 + 75, img_size[1]],
         [img_size[0]/2 + 60, img_size[1]/2 + 100]])

dst = np.float32(
        [[img_size[0]/4, 0],
         [img_size[0]/4, img_size[1]],
         [img_size[0]*3/4, img_size[1]],
         [img_size[0]*3/4, 0]])

In [6]:
# Get M and Minv for perspective tranfsorm
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

In [7]:
# Define perspective transform function
def warper2(img, M):
    img_size = (img.shape[1], img.shape[0])
    #M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size,cv2.INTER_NEAREST)
    return warped

In [8]:
def sliding_window(binary_warped):
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 70
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit

In [9]:
def selection_window(binary_warped, left_fit, right_fit):
    
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 60
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
     
    return left_fit, right_fit

In [10]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

In [11]:
def cal_curvature(ploty, left_fitx, right_fitx):
    y_eval = np.max(ploty)
    center_fitx = (left_fitx + right_fitx) / 2.0
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    center_fit_cr = np.polyfit(ploty*ym_per_pix, center_fitx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    center_curverad = ((1 + (2*center_fit_cr[0]*y_eval*ym_per_pix + center_fit_cr[1])**2)**1.5) / np.absolute(2*center_fit_cr[0])
    
    return left_curverad, right_curverad, center_curverad

In [12]:
def cal_car_offset(left_fitx, right_fitx):
    center_fitx = (left_fitx[-1] + right_fitx[-1]) / 2.0
    offset = (midpoint - center_fitx) * xm_per_pix
    
    return offset

In [13]:
def project_lane_lines(undstimg,left_fitx,right_fitx,yvals,Minv):
    
    # Create an image to draw the lines on
    color_warp = np.zeros_like(undstimg).astype(np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, yvals]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, yvals])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    cv2.polylines(color_warp, np.int32([pts_left]), isClosed=False, color=(255,0,255), thickness=15)
    cv2.polylines(color_warp, np.int32([pts_right]), isClosed=False, color=(0,255,255), thickness=15)
    #undist = undistort(img)    
    #unwarp,Minv = warp(img,tobird=False)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undstimg.shape[1], undstimg.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undstimg, 1, newwarp, 0.3, 0)
    return result

In [14]:
def draw_info(img, left_curverad, right_curverad, center_curverad, car_offset):
    new_img = np.copy(img)
    img_h = new_img.shape[0]
    font = cv2.FONT_HERSHEY_DUPLEX
    
    text = 'Radius of carvature : ' + '{:04.2f}'.format(center_curverad) + 'm'
    cv2.putText(new_img, text, (40,70), font, 1.5, (200,255,155), 2, cv2.LINE_AA)
    
    direction = ''
    if car_offset > 0:
        direction = 'right'
    elif car_offset < 0:
        direction = 'left'
        
    abs_center_dist = abs(car_offset)
    text = 'Vehicle is ' + '{0:.2}'.format(np.absolute(car_offset)) + 'm ' + direction + ' of center'
    cv2.putText(new_img, text, (40,120), font, 1.5, (200,255,155), 2, cv2.LINE_AA)
    return new_img

In [15]:
left_fit = []
right_fit = []
selection_window_enable = False

def process_image(img):
    
    # correct and undistort camera image
    undist_img = undist_image(img, mtx, dist)
    
    # convert to combined binay image (s_thresh=(170, 255), sx_thresh=(20, 100))
    color_binary, combined_binary = pipeline(undist_img, (170, 255), (20, 100))
    
    # warp
    binary_warped = warper2(combined_binary, M)
    
    global left_fit
    global right_fit
    global selection_window_enable

    # window for finding lane
    #prev_lfit = left_fit
    #prev_rfit = right_fit
    if selection_window_enable == False:
        left_fit, right_fit = sliding_window(binary_warped)
        selection_window_enable = True
    else:
        #left_fit, right_fit = sliding_window(binary_warped)
        left_fit, right_fit = selection_window(binary_warped,left_fit, right_fit)
    
    # cal x y fit
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # cal curvature and car offset
    left_curverad, right_curverad, cernter_curverad = cal_curvature(ploty, left_fitx, right_fitx)
    
    # cal car offset
    car_offset = cal_car_offset(left_fitx, right_fitx)
    
    # project on undistorted image
    projected_img = project_lane_lines(undist_img, left_fitx, right_fitx, ploty, Minv)
    final_img = draw_info(projected_img, left_curverad, right_curverad, cernter_curverad, car_offset)
    
    return final_img

In [16]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [17]:
project_video = 'project_video_ks.mp4'
clip1 = VideoFileClip('project_video.mp4')
project_video_clip = clip1.fl_image(process_image)
%time project_video_clip.write_videofile(project_video, audio=False)

[MoviePy] >>>> Building video project_video_ks.mp4
[MoviePy] Writing video project_video_ks.mp4


100%|█████████▉| 1260/1261 [05:15<00:00,  3.99it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_ks.mp4 

CPU times: user 8min 28s, sys: 54.3 s, total: 9min 22s
Wall time: 5min 18s
